In [3]:
from pyarrow import json
import os
import sys
import math
import datetime
import pathlib
import pyarrow.parquet as pq
import pyarrow.orc as po
import time
dir_path = pathlib.Path(os.path.abspath('')).resolve()
print(dir_path)
HOME_DIR = str(dir_path).split('/OpenFormat')[0]

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

PROJ_SRC_DIR = f'{HOME_DIR}/OpenFormat'
sys.path.insert(1, f'{PROJ_SRC_DIR}')
from python.scripts.utils import *
from pyarrow import json

pq_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/pq_default.json')
orc_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/orc_default.json')
pq_param = pq_config[0]
orc_param = orc_config[0]

/mnt/OpenFormat/python/nested
loading config from /mnt/OpenFormat/python/experiments/pq_default.json ...
loading config from /mnt/OpenFormat/python/experiments/orc_default.json ...


In [7]:
# assign directory
directory = './'
 
for root, dirs, files in os.walk(directory):
    for sample_dir in dirs:
        if sample_dir.startswith('sample') == False:
            continue
        for root, dirs, files in os.walk(sample_dir):
            for filename in files:
                json_name = os.path.join(root, filename)
                if json_name.endswith('.json') == False:
                    continue
                base_name = json_name.split('.')[0]
                try:
                    table = json.read_json(json_name)
                    pq.write_table(table, f'{base_name}.parquet', **pq_param)
                    po.write_table(table, f'{base_name}.orc', **orc_param)
                except:
                    print(f'{filename} failed to parse')

movies.json failed to parse
users.json failed to parse
shipwrecks.json failed to parse
trips.json failed to parse
companies.json failed to parse
inspections.json failed to parse
tweets.json failed to parse
stories.json failed to parse
routes.json failed to parse


In [8]:
pq_size_sum = 0
orc_size_sum = 0
for root, dirs, files in os.walk(directory):
    for sample_dir in dirs:
        if sample_dir.startswith('sample') == False:
            continue
        for root, dirs, files in os.walk(sample_dir):
            for filename in files:
                name = os.path.join(root, filename)
                if name.endswith('.parquet'):
                    pq_size_sum += os.path.getsize(name)
                elif name.endswith('.orc'):
                    orc_size_sum += os.path.getsize(name)
print(f'pq_size_sum: {pq_size_sum}')
print(f'orc_size_sum: {orc_size_sum}')

pq_size_sum: 147151212
orc_size_sum: 130338778


In [4]:
from pyarrow import json
json_name = 'arxiv-metadata-oai-snapshot.json'
base_name = json_name.split('.json')[0]
try:
    table = json.read_json(json_name)
    pq.write_table(table, f'{base_name}.parquet', **pq_param)
    po.write_table(table, f'{base_name}.orc', **orc_param)
except Exception as e:
    print(e)
    print(f'{json_name} failed to parse')
print(f'pq_size: {os.path.getsize(f"{base_name}.parquet")}')
print(f'orc_size: {os.path.getsize(f"{base_name}.orc")}')

pq_size: 1766858930
orc_size: 1676926899


In [31]:
os.system('sync; echo 3 | sudo tee /proc/sys/vm/drop_caches')

3


0

In [32]:
%time
tmp_table = pq.read_table(f'{base_name}.parquet', 
              use_threads=False, pre_buffer=False, columns=['versions'])
# print(tmp_table)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 8.82 µs


In [30]:
%time
tmp_table = po.read_table(f'{base_name}.orc',  columns=['versions'])
# print(tmp_table)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.82 µs


In [18]:
# load json from dblp.v12.json and convert it into newline separated json
import json as native_json
js = native_json.load(open('dblp.v12.json'))
with open('dblp.v12.jsonl', 'w') as f:


: 

: 

In [11]:
print(table.schema)

id: string
submitter: string
authors: string
title: string
comments: string
journal-ref: string
doi: string
report-no: string
categories: string
license: string
abstract: string
versions: list<item: struct<version: string, created: string>>
  child 0, item: struct<version: string, created: string>
      child 0, version: string
      child 1, created: string
update_date: timestamp[s]
authors_parsed: list<item: list<item: string>>
  child 0, item: list<item: string>
      child 0, item: string


In [15]:
os.system('sync; echo 3 | sudo tee /proc/sys/vm/drop_caches')

3


0

In [16]:
%time
begin = time.time()
tmp_table = pq.read_table(f'./experiments/depth_size/62.parquet', 
              use_threads=False, pre_buffer=False)
print(f'pq time: {time.time() - begin}')
# print(tmp_table)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 7.63 µs
pq time: 1.037780523300171


In [17]:
%time
begin = time.time()
tmp_table = po.read_table(f'./experiments/depth_size/62.orc')
print(f'orc time: {time.time() - begin}')

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 7.39 µs
orc time: 3.3405394554138184
